## Fixed Data:

In [1]:
pip install fredapi

Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import requests
import pandas as pd
from fredapi import Fred

API_KEY = '1vGb0FOsg2hEIGrEhpueLCcWd1Y'
FRED_API_KEY = 'b13a905b98b5dfc1f3910d6a0d875ae3'

rootUrl = 'https://api.glassnode.com'

symbolList = ['BTC', 'ETH', 'LTC', 'AAVE', 'ABT', 'AMPL', 'ANT', 'ARMOR', 'BADGER', 'BAL', 'BAND', 'BAT', 'BIX', 'BNT', 'BOND', 'BRD', 'BUSD', 'BZRX', 'CELR', 'CHSB', 'CND', 'COMP', 'CREAM', 'CRO', 'CRV', 'CVC', 'CVP', 'DAI', 'DDX', 'DENT', 'DGX', 'DHT', 'DMG', 'DODO', 'DOUGH', 'DRGN', 'ELF', 'ENG', 'ENJ', 'EURS', 'FET', 'FTT', 'FUN', 'GNO', 'GUSD', 'HEGIC', 'HOT', 'HPT', 'HT', 'HUSD', 'INDEX', 'KCS', 'LAMB', 'LBA', 'LDO', 'LEO', 'LINK', 'LOOM', 'LRC', 'MANA', 'MATIC',
              'MCB', 'MCO', 'MFT', 'MIR', 'MKR', 'MLN', 'MTA', 'MTL', 'MX', 'NDX', 'NEXO', 'NFTX', 'NMR', 'Nsure', 'OCEAN', 'OKB', 'OMG', 'PAY', 'PERP', 'PICKLE', 'PNK', 'PNT', 'POLY', 'POWR', 'PPT', 'QASH', 'QKC', 'QNT', 'RDN', 'REN', 'REP', 'RLC', 'ROOK', 'RPL', 'RSR', 'SAI', 'SAN', 'SNT', 'SNX', 'STAKE', 'STORJ', 'sUSD', 'SUSHI', 'TEL', 'TOP', 'UBT', 'UMA', 'UNI', 'USDC', 'USDK', 'USDP', 'USDT', 'UTK', 'VERI', 'WaBi', 'WAX', 'WBTC', 'WETH', 'wNXM', 'WTC', 'YAM', 'YFI', 'ZRX']

symbolListShort = ['BTC', 'ETH', 'LTC']

endPoints = {
    'Price': '/v1/metrics/market/price_usd_close',
    'MarketCap': '/v1/metrics/market/marketcap_usd',
    'TransactionCount': '/v1/metrics/transactions/count',
    'ActiveAddress': '/v1/metrics/addresses/active_count',
}

## <font color='red'> It seems the risk-free rate is daily rate. is it in percentage or in unit 1? need to adjust to weekly rate and select Monday

In [4]:
fred = Fred(api_key=FRED_API_KEY)
Risk_free_rate = fred.get_series('DGS1MO')
Risk_free_rate = Risk_free_rate.to_frame()
Risk_free_rate = Risk_free_rate.reset_index()
Risk_free_rate = Risk_free_rate.rename(columns={'index':'Date', 0:'RiskFree'})

## Get Multiple Attributes for Multiple Coins:

In [5]:
### Define a function to pull data according to input values(coin, feature)
### After running this method, it should return pulled data, and also store it in memory named as "coin_feature"

def PullData(coin, feature):
    print(f"Pulling {coin}'s {feature} data")

    # Assemble full request url and get responses:
    reqUrl = rootUrl + endPoints[feature]
    res = requests.get(reqUrl,
                       params={'a': coin, 'api_key': API_KEY, 'i':'1w'})
    data_raw = pd.read_json(res.text, convert_dates=['t'])

    # Rename column "v" according to features
    data = data_raw.rename(columns={"v" : f"{feature}", 't':'Date'})

    # Turn renamed dataframe into a global variable for future using
    name = f"{coin}_{feature}"
    globals()[name] = data

    # Return a dataframe
    return data

In [70]:
### Get all features(listed in endPoints) of all coins(listed in symbolListShort) 

from functools import reduce

for i in symbolListShort:
    print(f"Start Pulling {i}'s data:")

    # Preparation for merging
    frameName = f"{i}_Frames"
    locals()[frameName] = []

    # Get data
    for j in list(endPoints.keys()):
        locals()[frameName].append(PullData(i, j))

    # Merge to single dataframe
    print('Merging Dataframes...')
    dfName = f"{i}_All"
    # change merge method to outer to ensure not missing any values.
    locals()[dfName] = reduce(lambda  left,right: pd.merge(left,right, on='Date', how='outer'), locals()[frameName]) 
    
    # change merege method to left, to ensure all the records in api stored.
    locals()[dfName] = locals()[dfName].merge(Risk_free_rate,how = 'left').sort_values(by=['Date'])

    print(f"{i} data successfully pulled\n")
print('\nALL DATA SUCCESSFULLY PULLED!')

Start Pulling BTC's data:
Pulling BTC's Price data
Pulling BTC's MarketCap data
Pulling BTC's TransactionCount data
Pulling BTC's ActiveAddress data
Merging Dataframes...
BTC data successfully pulled

Start Pulling ETH's data:
Pulling ETH's Price data
Pulling ETH's MarketCap data
Pulling ETH's TransactionCount data
Pulling ETH's ActiveAddress data
Merging Dataframes...
ETH data successfully pulled

Start Pulling LTC's data:
Pulling LTC's Price data
Pulling LTC's MarketCap data
Pulling LTC's TransactionCount data
Pulling LTC's ActiveAddress data
Merging Dataframes...
LTC data successfully pulled


ALL DATA SUCCESSFULLY PULLED!


## <font color='red'> Imputation for NA values -- to be resolved

## Calculate the return - Now only pick lastest 52 records for testing

In [71]:
nainfo = pd.DataFrame(index=LTC_All.columns.values)  #can be improved

for i in symbolListShort:
     
    dfName = f"{i}_All" 

    temp = locals()[dfName][-53:-1].reset_index()
    
    locals()[dfName] = locals()[dfName][-52:].reset_index()   
    
    natemp = locals()[dfName].isnull().sum().to_frame(name= i)
    
    #nainfo pick all the na values
    nainfo = nainfo.join(natemp)  #can be removed, just for data checking purpose
    
    
    locals()[dfName]['Return'] = (locals()[dfName]['Price'] - temp['Price'])/temp['Price']
    


In [74]:
LTC_All.head()

,index,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree,Return
0,415,2021-04-26,269.060433,1.817989e+10,777669,1261380,0.02,0.201887
1,416,2021-05-03,385.704466,2.608153e+10,882361,1438322,0.02,0.433524
2,417,2021-05-10,296.493781,2.006374e+10,884372,1530001,0.02,-0.231293
3,418,2021-05-17,144.373194,9.776754e+09,788793,1305682,0.00,-0.513065
4,419,2021-05-24,170.925525,1.158400e+10,719597,1180854,0.01,0.183915


## Calculate market index return for each week

In [184]:
## find market portfolio

markdf = pd.DataFrame(columns = LTC_All.columns.values)  # can be improved

markdf['Asset'] = ''

## step 1: pick all maket cap > 10000
for i in symbolListShort:
     
    dfName = f"{i}_All" 
    
    ## find all date with MarketCap greater than 10000
    temp = locals()[dfName].query("MarketCap > 1000000")
    temp['Asset'] = i
    markdf = markdf.append(temp)

## sort all the market cap and pick top 100
marketport = markdf.groupby(['Date']).apply(lambda x: x.nlargest(100,['MarketCap'])).reset_index(drop=True)

## markdf is a big dataframe include all crypto currencies makcap greater than 1000000
marketport = markdf.groupby('Date')['MarketCap'].sum().reset_index(name='TotalMarketCap')

marketInd = markdf.copy()
marketInd['weights'] = marketInd['Return'] * marketInd['MarketCap']
recomp = marketInd.groupby('Date')['weights'].sum().reset_index(name='TotalMarketReturn')
marketport['MarketIndexReturn'] = recomp['TotalMarketReturn']/ marketport['TotalMarketCap']



In [185]:
marketport = marketport.merge(Risk_free_rate, on='Date')
marketport['MKT'] = marketport['MarketIndexReturn'] - marketport['RiskFree']
MKT_f = marketport[['Date','MKT']]

In [188]:
MKT_f

,Date,MKT
0,2021-04-26,0.164346
1,2021-05-03,0.103098
2,2021-05-10,-0.187060
3,2021-05-17,-0.294532
4,2021-05-24,0.044466
5,2021-05-31,NaN
6,2021-06-07,0.032184
7,2021-06-14,-0.100472
8,2021-06-21,-0.095009
9,2021-06-28,0.025024


## <font color='green'>Testing purpose : try to find the first date with available price

In [89]:
df = pd.DataFrame(columns = LTC_All.columns.values)
df['Asset'] = ''

for i in symbolList:
    
 
    dfName = f"{i}_All" 
    test=pd.DataFrame
    df1 = locals()[dfName][locals()[dfName].Price.notnull()].iloc[0]
    
    df1['Asset'] = i
    df=df.append(df1, ignore_index=True)

    

In [90]:
df.to_csv("firstavailable_price.csv")

In [108]:
new = df.sort_values(by=['Date'])

In [78]:
LTC_All.Price.loc[~LTC_All.Price.isnull()]

0        3.149100
1        3.150000
2        2.830000
3        2.348333
4        2.150000
          ...    
462    129.241438
463    128.926292
464    111.343007
465    109.018201
466    104.648982
Name: Price, Length: 467, dtype: float64